### Librerias y datos

In [3]:
# Librerias
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
from importlib import reload

# Libreria con las funciones personalizadas
import creditScoring_toolKit
reload(creditScoring_toolKit)
from creditScoring_toolKit import *

In [4]:
# Cargamos datos
datos = pd.read_csv('Data/loan_data.csv')

### Preparacion de base

In [5]:
# Sustituimos valores NAN por -1 en caso de variables numericas y "Sin informacion" en casto de categoricas
for variable in datos.columns:
    if datos[variable].dtype in (int,float):
        datos[variable] = datos[variable].fillna(-1)
    else:
        datos[variable] = datos[variable].fillna('Sin informacion')

### Creamos base train y test

##### En caso de aun no haber creado las bases train y test

In [6]:
# Dividimos la base en train y test
train, test = train_test_split(datos, train_size=0.7,random_state=123)

In [7]:
# Guardamos train y test para no volver a hacer la muestra mas adelante
train.to_csv('Data/train.csv',index=False)
test.to_csv('Data/test.csv',index=False)

##### En caso de ya haber creado las bases train y test

In [8]:
# Cargamos datos de csv
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

### Cálculo information value y WOEs variables

In [9]:
# Clasificamos variables en categoricas y numericas sin incluir variable target
numericas, categoricas = var_numericas_categoricas(train,['SK_ID_CURR','TARGET'])

##### Variables categoricas

In [10]:
# Calculamos IV y WOEs para todas las variables categoricas

# df vacios donde se almacenaran los resultados
iv_categoricas = pd.DataFrame()
woe_categoricas = pd.DataFrame()

# Aplicamos la funcion calculate_iv_cat a todas las variables categoricas
for variable in categoricas:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        iv, woe = calculate_iv_cat(base=train,variable=variable,target='TARGET')

    if iv_categoricas.empty:
        iv_categoricas = iv
        woe_categoricas = woe
    else:
        iv_categoricas = pd.concat([iv_categoricas,iv])
        woe_categoricas = pd.concat([woe_categoricas,woe])

# Eliminamos variables de apoyo
del(iv,woe,variable)

##### Variables numericas

In [11]:
# Calculamos IV y WOEs de variables numericas

# df vacios donde se almacenaran los resultados
iv_numericas = pd.DataFrame()
woe_numericas = pd.DataFrame()

for variable in numericas:
    # Calcula las categorias optimas de cada variable numerica
    categorias = optimal_binning(train,variable,'TARGET')
    
    iv, woe = calculate_iv_num(train,variable,'TARGET',categorias)

    if iv_numericas.empty:
        iv_numericas = iv
        woe_numericas = woe
    else:
        iv_numericas = pd.concat([iv_numericas,iv])
        woe_numericas = pd.concat([woe_numericas,woe])

# Eliminamos variables de apoyo
del(iv,woe,variable)

##### Unimos resultados de numericas y categoricas

In [12]:
# Unimos resultados de IV y WOEs
# Information value
resultados_iv = pd.concat([iv_numericas,iv_categoricas])

# Information woe
resultados_woe = pd.concat([woe_numericas,woe_categoricas])

del(iv_numericas,iv_categoricas,woe_numericas,woe_categoricas)

### Base WOEs train y test

In [13]:
# Lista variables
variables = train.columns.to_list()
variables.remove('SK_ID_CURR')
variables.remove('TARGET')

In [21]:
# Creamos la base WOEs train
base_woes_train = pd.DataFrame()
for variable in variables:
    # Asignamos los valores woes de acuerdo al valor de la variable
    base_woe_variable = base_woes(train,variable,resultados_woe)
    
    # Agrupamos variables con valor woe
    if base_woes_train.empty:
        base_woes_train = base_woe_variable
    else:
        base_woes_train = pd.concat([base_woes_train,base_woe_variable],axis=1)

KeyboardInterrupt: 

In [15]:
# Creamos la base WOEs test
base_woes_test = pd.DataFrame()
for variable in variables:
    # Asignamos los valores woes de acuerdo al valor de la variable
    base_woe_variable = base_woes(test,variable,resultados_woe)
    
    # Agrupamos variables con valor woe
    if base_woes_test.empty:
        base_woes_test = base_woe_variable
    else:
        base_woes_test = pd.concat([base_woes_test,base_woe_variable],axis=1)

### Definimos bases X_train, X_test, y_train, y_test
A pesar de tener ya una base train y una base test, creamos estos cuatro DataFrames ya que es el estandar mas usado al momento de entrenar modelos.

In [22]:
# Creamos bases

# X_train
X_train = base_woes_train

# X_test
X_test = base_woes_test

# y_train
y_train = train['TARGET']

# y_test
y_test = test['TARGET']

### Modelo logistico

In [19]:
# Librerias
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

##### Modelo saturado (todas las variables)
Usualmente se calcula un modelo saturado con todas las variables disponibles, esto para darse una idea de cuales son las metricas (AUC, precision) a las que podemos aspirar si usaramos toda la informacion disponible.

In [23]:
# Modelo saturado con lista_variables seleccionadas
lista_variables = X_train.columns

# Entrenamos modelo y hacemos predicciones
model = LogisticRegression(max_iter=1000)
model.fit(X_train[lista_variables], y_train)
y_pred = model.predict(X_test[lista_variables])

# Calculamos metricas
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test[lista_variables])[:, 1])

# Imprimir los resultados
print(f"Exactitud del modelo test: {accuracy:.4f}")
print(f"Matriz de confusión test:\n{conf_matrix}")
print(f"AUC-ROC test: {roc_auc:.4f}")


y_pred = model.predict(X_train[lista_variables])
# Calculamos metricas
accuracy = accuracy_score(y_train, y_pred)
conf_matrix = confusion_matrix(y_train, y_pred)
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train[lista_variables])[:, 1])

# Imprimir los resultados
print(f"Exactitud del modelo train: {accuracy:.4f}")
print(f"Matriz de confusión train:\n{conf_matrix}")
print(f"AUC-ROC train: {roc_auc:.4f}")

modelo1_RL = model

Exactitud del modelo test: 0.9182
Matriz de confusión test:
[[84653    64]
 [ 7483    54]]
AUC-ROC test: 0.7432
Exactitud del modelo train: 0.9196
Matriz de confusión train:
[[197826    143]
 [ 17157    131]]
AUC-ROC train: 0.7465
